# $\Delta$ Class Development
Interface notebook testing a Python class created from `advanced_profile_creation-Dev`

### Author
John Wall (jwall@Dewberry.com)

### Import libraries

In [1]:
# Sci Stack
import pandas as pd
from matplotlib import pyplot as plt

# GeoSpatial Stack
import geopandas as gpd

import shapely
from shapely import geometry, ops
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points

import rasterio
from rasterio.plot import show
from rasterstats import zonal_stats, point_query

In [97]:
class DepthToInundationPts(object):
    """Geospaital points representing the cartographic location of
        lowest elevation (i.e. channel) at a transportation-route-stream
        intersection. The point is attributed with the difference in
        elevation between the top of the transporation route and
        the channel elevation.
        
       Parameters:
        streams (str): Path to streams as vectors
        transit_routes (str): Path to roads as vectors
        dem (str): Path to a digitial elevation model as raster
        prj (dict): Projection information
    """
    def __init__(self, streams, transit_routes, dem, prj):
        
        def load_data(data, prj, data_type):
            """Loads data into the object based on dataset type (i.e.
                streams, roads, etc.). Can easily be extended to
                include other types.
            """
            rprj_data = gpd.read_file(data).to_crs(prj)
            if data_type == "streams":
                return rprj_data
            elif data_type == "roads":
                return rprj_data.dissolve(by='FULL_STREE')
            else:
                print("Neither streams nor roads.")

        def find_intersections(self):
            """Finds intersections between *A* stream and
                transportation routes.

                THIS FUNCTION NEEDS TO BE REWRITTEN TO EXECUTE OVER ALL
                STREAMS OF INTEREST.
            """
            stream_shape = self._streams.geometry[0]
            routes = self._routes
            possible_intersections = routes.geometry.apply(lambda row: stream_shape.intersection(row))
            return possible_intersections[~possible_intersections.is_empty]
        
        def clip_transit_routes(self):
            """Clips transportation routes, converts them to single
                parts,explodes the results to account for routes
                with the same name. Contains logic to ensure only
                records with geospatial data are return.
            """
            clp_routes = self._intxn_polys.intersection(self._routes)

            single_lines = {}
            for i, transit in enumerate(clp_routes):
                if 'GeometryCollection' not in type(transit).__name__ :
                    if type(transit) is shapely.geometry.multilinestring.MultiLineString:
                        single_lines[clp_routes.index[i]] = ops.linemerge(transit)
                    else:
                        single_lines[clp_routes.index[i]] = transit

            columns=['transit','geometry']
            explode_lines = gpd.GeoDataFrame(single_lines.items(), columns=columns)
            try:
                return explode_lines.explode().droplevel(0).reset_index(0, drop=True)
            except:
                return explode_lines
        
        def create_offsets(self):
            """Creates left and right offset profile lines from clipped
                transportation routes.
            """
            gdf = self._clp_roads.copy()
            
            df_list = []
            for side in ['left', 'right']:
                gdf['side'] = side
                gdf['offset'] = gdf.geometry.apply(lambda road: road.parallel_offset(100, side, resolution=1))
                df_list.append(gdf.copy())
            return pd.concat(df_list).reset_index(0, drop=True)
        
        def create_topographic_profiles(self, column):
            """Updates 2d lines to 3d topographic profiles"""
            lns = self._offset_lines
            for i in lns.index:
                profile_points = point_query(lns[column][i], dem)[0]
                points3d = []
                for ii, z in enumerate(profile_points):
                    x, y = list(lns[column][i].coords)[ii]
                    points3d.append(Point([x,y,z]))
                lns.loc[i][column] = LineString(points3d)
            return lns
        
        # Basic, input properties
        self._streams = load_data(streams, prj, "streams")
        self._routes = load_data(transit_routes, prj, "roads")
        self._dem = dem
        
        # Computed properties
        self._intxn_points = find_intersections(self)
        self._intxn_polys = self._intxn_points.buffer(100)
        self._clp_roads = clip_transit_routes(self)
        self._offset_lines = create_offsets(self)
        self._offset_profiles = create_topographic_profiles(self, 'offset')
        self._route_profiles = create_topographic_profiles(self, 'geometry')
    
    @property
    def gdf_streams(self):
        """GeoPandas GeoDataFrame of Streams"""
        return self._streams
    
    @property
    def gdf_routes(self):
        """GeoPandas GeoDataFrame of Transportation routes"""
        return self._routes
    
    @property
    def dem(self):
        """Digitial Elevation Model used to cross section creation"""
        return self._dem
    
    @property
    def intersection_points(self):
        """Stream and transportation intersection points"""
        return self._intxn_points
    
    @property
    def intersection_polygons(self):
        """Buffers around intersection points"""
        return self._intxn_polys
    
    @property
    def clipped_roads(self):
        """Transportation routes clipped to buffers"""
        return self._clp_roads
    
    @property
    def offset_lines(self):
        """Lines offset from the clipped transportation route"""
        return self._offset_lines
    
    @property
    def offset_profiles(self):
        """Offset topographic profiles"""
        return self._offset_profiles
    
    @property
    def route_profiles(self):
        """Road topographic profiles"""
        return self._route_profiles

In [98]:
streams = "/mnt/c/gis/fcast_data/sample_streams.shp"
roads = "/mnt/c/gis/fcast_data/sample_roads.shp"
dem = "/mnt/c/gis/fcast_data/tiffs/c67567_aoi.tif"

our_prj = {'proj': 'aea', 'lat_1': 20, 'lat_2': 60, 'lat_0': 40,
             'lon_0': -96, 'x_0': 0, 'y_0': 0, 'ellps': 'GRS80',
             'units': 'm', 'no_defs': True}

In [99]:
datasets = DepthToInundationPts(streams, roads, dem, our_prj)

In [100]:
datasets.offset_profiles.geometry[0].coords[0]

(1264274.342034306, -452328.8154376416, 233.562630744969)